In [ ]:
#auto reload 
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd
import numpy as np
import os 
from datasets import load_dataset, load_from_disk


In [ ]:
import pandas as pd 
 
bow_english = pd.read_csv("data/unigram_freq.csv")

In [ ]:
wmt14 = load_dataset("wmt14", "fr-en", data_dir="data/")

# Accessing example data
train_data = wmt14["train"]
val_data = wmt14["validation"]


In [ ]:
examples = 4 
#check some data and their translation
for i, t in zip(range(examples), train_data):
    ex = t["translation"]
    print(ex["fr"], " -> ", ex["en"], "\n\n")

In [ ]:
from sacremoses import MosesTokenizer, MosesDetokenizer

# Sample sentence
sentence = "This is an  example sentence with words that may not be in the shortlist."

mt  = MosesTokenizer(lang="en")

tokenized = mt.tokenize(sentence)


In [ ]:
tokenized

In [ ]:
class TokenizerWrapper:
    def __init__(self, tokenizer):
        self.tokenizer = tokenizer
    
    def tokenize_function(self, examples):
        return {"tokenized" : self.tokenizer.tokenize( examples["translation"])}
    
tokenizer_wrapper = TokenizerWrapper(mt)


In [ ]:
if not os.path.exists("data/tokenized_train_data"):
    tokenized_train_data  = train_data.map(tokenizer_wrapper.tokenize_function, batched=True, num_proc=19, remove_columns=["translation"])
    tokenized_train_data.save_to_disk("data/tokenized_train_data")


In [ ]:
    
if not os.path.exists("data/tokenized_val_data"):
    tokenized_val_data  = val_data.map(tokenizer_wrapper.tokenize_function, batched=True, num_proc=19, remove_columns=["translation"])
    tokenized_val_data.save_to_disk("data/tokenized_val_data")


In [ ]:
tokenized_train_data = load_from_disk("data/tokenized_train_data")
tokenized_val_data = load_from_disk("data/tokenized_val_data")

In [ ]:
most_frequent_english_words = bow_english["word"][:30000].apply(lambda x: str(x)).tolist()
most_frequent_english_words.append("UNK")
tokenized_most_frequent_english_words = mt.tokenize(" ".join(most_frequent_english_words))

In [ ]:
class toIdTransform:
    def __init__(self, most_frequent_words):
        self.most_frequent_words = most_frequent_words
        self.word_to_id = {word: i for i, word in enumerate(most_frequent_words)}
    
    def __call__(self, tokenized):
        return {"ids" : [self.word_to_id.get(word, self.word_to_id["UNK"]) for word in tokenized["tokenized"]]}
    
to_id_transform = toIdTransform(tokenized_most_frequent_english_words)


In [ ]:
if not os.path.exists("data/id_train_data"):
    tokenized_train_data = tokenized_train_data.map(to_id_transform, batched=True, num_proc=19)
    tokenized_train_data.save_to_disk("data/id_train_data")


In [ ]:
if not os.path.exists("data/id_val_data"):
    tokenized_val_data = tokenized_val_data.map(to_id_transform, batched=True, num_proc=19)
    tokenized_val_data.save_to_disk("data/id_val_data")

In [ ]:
tokenized_train_data = load_from_disk("data/tokenized_id_train_data")
tokenized_val_data = load_from_disk("data/tokenized_id_val_data")

In [ ]:
import torch 

def id_to_vec(ids: torch.tensor, vocab_size: int):
    vec = torch.zeros(ids.shape[0], vocab_size)
    vec[torch.arange(ids.shape[0]), ids] = 1
    return vec

train_ids = torch.tensor(tokenized_train_data["ids"])   
val_ids = torch.tensor(tokenized_val_data["ids"])

train_vecs = id_to_vec(train_ids, len(tokenized_most_frequent_english_words))
val_vecs = id_to_vec(val_ids, len(tokenized_most_frequent_english_words))
